In [ ]:
#安装步骤
conda activate keras220CpuJupyter
conda install tensorflow==1.10
conda install keras=2.2
conda install pydot
git clone git@github.com:lukeliuli/hierarchical-multilabel-classification.git

In [ ]:
#程序正常时运行
!pwd
!python train_model_hmcnf.py

In [13]:
#调试程序
#1.读入数据
from arff_reader import ArffReader
import numpy as np
import model_hmcnf

file_name1 = 'cellcycle/cellcycle_FUN.train.arff'

x = []
y = []

for row in ArffReader(file_name1):
    x.append(row[0:-1])
    y.append(row[-1])

x = np.array(x)
y = np.array(y)

print(x.shape)
print(y.shape)

# hierarchy-sizes [18, 80, 178, 142, 77, 4, 0, 0], sum=499
# (1628, 77)
# (1628, 499


hierarchy = [18, 80, 178, 142, 77, 4]
features_size = x.shape[1]
label_size = y.shape[1]
beta = 0.5
dropout_rate=0.1
relu_size=384


(1628, 77)
(1628, 499)


ValueError: Output tensors to a Model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: Tensor("add_6/add:0", shape=(?, 499), dtype=float32)

In [42]:
#建立HMCNF模型
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model

def local_model(num_labels, dropout_rate, relu_size):
    model = tf.keras.Sequential()
    model.add(layers.Dense(relu_size, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_labels, activation='sigmoid'))
    return model


def global_model(dropout_rate, relu_size):
    model = tf.keras.Sequential()
    model.add(layers.Dense(relu_size, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    return model


def sigmoid_model(label_size):
    model = tf.keras.Sequential()
    model.add(layers.Dense(label_size, activation='sigmoid',name="global"))
    return model

features = layers.Input(shape=(features_size,))
global_models = []
local_models = []

for i in range(len(hierarchy)):
    if i == 0:
        global_models.append(global_model(dropout_rate, relu_size)(features))
    else:
        global_models.append(global_model(dropout_rate, relu_size)(layers.concatenate([global_models[i-1], features])))

p_glob = sigmoid_model(label_size)(global_models[-1])


#显示只有全局模型的情况
#modelTmp1 = tf.keras.Model(inputs=[features], outputs=[p_glob])
#modelTmp1.summary()#
#plot_model(modelTmp1, to_file='Flatten1.png', show_shapes=True)

for i in range(len(hierarchy)):
    local_models.append(local_model(hierarchy[i], dropout_rate, relu_size)(global_models[i]))
    
#显示只有局部局模型的情况(部分全局)
p_loc = layers.concatenate(local_models)
#modelTmp2 = tf.keras.Model(inputs=[features], outputs=[p_loc])
#modelTmp2.summary()#
#plot_model(modelTmp2, to_file='Flatten2.png', show_shapes=True)
p_glob1 = layers.Lambda(lambda x: x*beta,name="global")(p_glob)
p_loc1 = layers.Lambda(lambda x: x*(1-beta),name="local")(p_loc)

labels = layers.add([p_glob1, p_loc1])

model = tf.keras.Model(inputs=[features], outputs=[labels])
plot_model(model, to_file='FlattenAll.png', show_shapes=True)


model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),loss='binary_crossentropy',metrics=['mae'])

model.fit([x],[y],epochs=100, batch_size=256)

model.save("hmcnf.h5")

Epoch 1/100
1628/1628 [==============================] - 7s 4ms/step - loss: 0.5729 - mean_absolute_error: 0.4289
Epoch 2/100
1628/1628 [==============================] - 1s 326us/step - loss: 0.1164 - mean_absolute_error: 0.0573
Epoch 3/100
1628/1628 [==============================] - 1s 349us/step - loss: 0.1096 - mean_absolute_error: 0.0279
Epoch 4/100
1628/1628 [==============================] - 1s 319us/step - loss: 0.0971 - mean_absolute_error: 0.0372
Epoch 5/100
1628/1628 [==============================] - 1s 317us/step - loss: 0.0907 - mean_absolute_error: 0.0393
Epoch 6/100
1628/1628 [==============================] - 1s 315us/step - loss: 0.0865 - mean_absolute_error: 0.0359
Epoch 7/100
1628/1628 [==============================] - 1s 353us/step - loss: 0.0835 - mean_absolute_error: 0.0338
Epoch 8/100
1628/1628 [==============================] - 1s 317us/step - loss: 0.0812 - mean_absolute_error: 0.0341
Epoch 9/100
1628/1628 [==============================] - 1s 324us/step - l

1628/1628 [==============================] - 1s 317us/step - loss: 0.0338 - mean_absolute_error: 0.0210
Epoch 72/100
1628/1628 [==============================] - 1s 351us/step - loss: 0.0335 - mean_absolute_error: 0.0209
Epoch 73/100
1628/1628 [==============================] - 1s 324us/step - loss: 0.0330 - mean_absolute_error: 0.0206
Epoch 74/100
1628/1628 [==============================] - 1s 311us/step - loss: 0.0325 - mean_absolute_error: 0.0205
Epoch 75/100
1628/1628 [==============================] - 1s 314us/step - loss: 0.0322 - mean_absolute_error: 0.0204
Epoch 76/100
1628/1628 [==============================] - 1s 355us/step - loss: 0.0317 - mean_absolute_error: 0.0200
Epoch 77/100
1628/1628 [==============================] - 1s 318us/step - loss: 0.0313 - mean_absolute_error: 0.0200
Epoch 78/100
1628/1628 [==============================] - 1s 319us/step - loss: 0.0310 - mean_absolute_error: 0.0197
Epoch 79/100
1628/1628 [==============================] - 1s 315us/step - los

In [44]:
!rm -rf *png *h5 